# Sentinel 2 Atmospheric Correction in Google Earth Engine

### Import modules 
and initialize Earth Engine

In [ ]:
import ee
import geemap
from Py6S import *
import datetime
import math
import os
import sys
from atmospheric import Atmospheric

ee.Initialize()

In [ ]:
#s = SixS()

#SixS.test()

### Function TOA to BOA

In [ ]:
# Spectral Response functions
def spectralResponseFunction(bandname):
    """
    Extract spectral response function for given band name
    """
    bandSelect = {
        'B1':PredefinedWavelengths.S2A_MSI_01,
        'B2':PredefinedWavelengths.S2A_MSI_02,
        'B3':PredefinedWavelengths.S2A_MSI_03,
        'B4':PredefinedWavelengths.S2A_MSI_04,
        'B5':PredefinedWavelengths.S2A_MSI_05,
        'B6':PredefinedWavelengths.S2A_MSI_06,
        'B7':PredefinedWavelengths.S2A_MSI_07,
        'B8':PredefinedWavelengths.S2A_MSI_08,
        'B8A':PredefinedWavelengths.S2A_MSI_8A,
        'B9':PredefinedWavelengths.S2A_MSI_09,
        'B10':PredefinedWavelengths.S2A_MSI_10,
        'B11':PredefinedWavelengths.S2A_MSI_11,
        'B12':PredefinedWavelengths.S2A_MSI_12,
        }

    return Wavelength(bandSelect[bandname])

# TOA Reflectance to Radiance
def toa_to_rad(bandname):
    """
    Converts top of atmosphere reflectance to at-sensor radiance
    """

    # solar exoatmospheric spectral irradiance
    ESUN = info['SOLAR_IRRADIANCE_'+bandname]
    solar_angle_correction = math.cos(math.radians(solar_z))

    # Earth-Sun distance (from day of year)
    doy = scene_date.timetuple().tm_yday
    d = 1 - 0.01672 * math.cos(0.9856 * (doy-4))
    # http://physics.stackexchange.com/
    # questions/177949/earth-sun-distance-on-a-given-day-of-the-year

    # conversion factor
    multiplier = ESUN*solar_angle_correction/(math.pi*d**2)

    # at-sensor radiance
    rad = toa.select(bandname).multiply(multiplier)

    return rad

# Radiance to Surface Reflectance
def surface_reflectance(bandname):
    """
    Calculate surface reflectance from at-sensor radiance given waveband name
    """

    # run 6S for this waveband
    s.wavelength = spectralResponseFunction(bandname)
    # execute 6S objects
    s.run()

    # extract 6S outputs
    Edir = s.outputs.direct_solar_irradiance                # irradiancia solar direct
    Edif = s.outputs.diffuse_solar_irradiance               # irradiancia solar difFusE
    Lp   = s.outputs.atmospheric_intrinsic_radiance         # path radiance
    absorb  = s.outputs.trans['global_gas'].upward          # absorption transmissivity
    scatter = s.outputs.trans['total_scattering'].upward    # scattering transmissivity
    tau2 = absorb*scatter                                   # total transmissivity

    # Note: s.outputs are calculated starting from 6S objects defined in `conversion` function

    # radiance to surface reflectance
    rad = toa_to_rad(bandname)
    ref = rad.subtract(Lp).multiply(math.pi).divide(tau2*(Edir+Edif))


    return ref

### Main function

In [ ]:
# Main function
def conversion(img,gpath):

    # write image date
    date = img.date()

    # Defining global variables:
    global toa
    global info
    global scene_date
    global solar_z
    global s
    
    # top of atmosphere reflectance
    toa = img.divide(10000)

    # METADATA
    info = img.getInfo()['properties']
    scene_date = datetime.datetime\
        .utcfromtimestamp(info['system:time_start']/1000)
    solar_z = info['MEAN_SOLAR_ZENITH_ANGLE']

    # ATMOSPHERIC CONSTITUENTS
    h2o = Atmospheric.water(geom,date).getInfo()
    o3 = Atmospheric.ozone(geom,date).getInfo()
    # Atmospheric Optical Thickness
    aot = Atmospheric.aerosol(geom,date).getInfo()

    # TARGET ALTITUDE (km)
    SRTM = ee.Image('CGIAR/SRTM90_V4')
    alt = SRTM.reduceRegion(reducer = ee.Reducer.mean(),
        geometry = geom.centroid()).get('elevation').getInfo()
    km = alt/1000 # i.e. Py6S uses units of kilometers
    
    # 6S OBJECT
    # Instantiate
    s = SixS()

    # Atmospheric constituents
    s.atmos_profile = AtmosProfile.UserWaterAndOzone(h2o,o3)
    s.aero_profile = AeroProfile.Continental
    s.aot550 = aot

    # Earth-Sun-satellite geometry
    s.geometry = Geometry.User()
    s.geometry.view_z = 0               # cálculo asumiendo vision en NADIR
    s.geometry.solar_z = solar_z        # ángulo cenital solar
    s.geometry.month = scene_date.month # mes usado en la distancia Earth-Sun
    s.geometry.day = scene_date.day     # día usado en la distancia Earth-Sun
    s.altitudes\
        .set_sensor_satellite_level()   # Altitud del sensor
    s.altitudes\
        .set_target_custom_altitude(km) # Altitud de la superficie

    # ATMOSPHERIC CORRECTION (by waveband)
    output = img.select('QA60')
    for band in ['B1','B2','B3','B4','B5','B6','B7','B8','B8A','B9','B10','B11','B12']:
             print(band)
             output = output.addBands(surface_reflectance(band))
             
    # set some properties for export
    dateString = scene_date.strftime("%Y-%m-%d")
    ref = output.set({'satellite':'Sentinel 2',
                   'fileID':info['system:index'],
                   'date':dateString,
                   'aerosol_optical_thickness':aot,
                   'water_vapour':h2o,
                   'ozone':o3})

    # define YOUR assetID or folder
    assetID = gpath + dateString

    # export
    export = ee.batch.Export.image.toAsset(\
         image=ref,
         description='sentinel2_atmcorr_export',
         assetId = assetID,
         region = region,
         crs = 'EPSG:4326',
         scale = 10)

   
    export.start()
    # print a message for each exported image
    return print("image "+ assetID +" exported")

# End of main function

### time and place
Define the time.

In [ ]:
# Define a year to calculate.
# year1 should be between 2016 to 2019 for our case of study.
# Create year1 = 2020 to apply Machine Learning model.
year1 = 2020
year2 = year1 + 1

start = ee.Date(str(year1) + '-10-01')
stop = ee.Date(str(year2) + '-01-01')

# Percentage cloud coverage
percentage = 20

### Region of study
The following code will determine the region where the analysis is apply

In [ ]:
# Rectangle with the ROI
geom = ee.Geometry.Rectangle(-70.8200199999999995, -33.7983520000000013, 
                             -70.8068049999999971, -33.7804399999999987)

region = geom.buffer(1000).bounds().getInfo()['coordinates']

### Filter the Sentinel 2 collection
The following code will grab the ImageCollection on the dates and ROI.

In [ ]:
S2 = ee.ImageCollection('COPERNICUS/S2')\
.filterBounds(geom)\
.filterDate(start, stop)\
.filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', percentage)\
.sort('system:time_start')\
.distinct('system:time_start')

# GEE path to save the image inside of a image collection (GEE account)
gpath = 'users/diegoalarcondiaz/sen2_cor/'

# List with images to filter
features = S2.getInfo()['features']
#print(features)

# Automatic correction
for i in features:
    #print(i)
    id = i['id']
    conversion(ee.Image(id),gpath)